In [1]:
from sciencecrawler.pubmed import PubmedSearch
import json
from neo4jsciencecrawler import Neo4jScienceCrawler
import time

In [2]:
words_covid = ["COVID-19","SARS‑CoV‑2","2019-nCoV","HCoV-19","hCoV-19","2019-nCoV","Wuhan coronavírus","Wuhan pneumonia","Pneumonia in Wuhan"]
words_hidro = ["Hydroxychloroquine","(Hydroxy)chloroquine","Chloroquine","CQ","HCQ"]


In [3]:
searchengine = PubmedSearch('chloroquine')

In [4]:
total = searchengine.total_search

In [5]:
total

28910

In [6]:
def check_article(article,words):
    print(article['title'])
    print(article['pubmed_url'])
    for word in words_covid:
        if word.lower() in article['title'].lower():
            return True
        if word.lower() in article['abstract'].lower():
            return True
    return False

In [7]:
searchengine.get_next_page()

In [8]:
articles = searchengine.get_list_articles()

https://pubmed.ncbi.nlm.nih.gov/8112211/
https://pubmed.ncbi.nlm.nih.gov/5027609/
https://pubmed.ncbi.nlm.nih.gov/3068119/
https://pubmed.ncbi.nlm.nih.gov/26687632/
https://pubmed.ncbi.nlm.nih.gov/33775077/
https://pubmed.ncbi.nlm.nih.gov/26365362/
https://pubmed.ncbi.nlm.nih.gov/4170972/
https://pubmed.ncbi.nlm.nih.gov/4912633/
https://pubmed.ncbi.nlm.nih.gov/22242848/
https://pubmed.ncbi.nlm.nih.gov/31701183/


In [10]:
searchengine.current_page

2

In [11]:
for article in articles:
    try:
        result = check_article(article.to_json(),words_covid)
        print(result)
        print(check_article(article.to_json(),words_hidro))
        print("====================")
    except Exception as ex:
        print("====================")
        print(article.title)
        print(str(ex))
        print("====================")

https://pubmed.ncbi.nlm.nih.gov/8112211/
https://pubmed.ncbi.nlm.nih.gov/5027609/
https://pubmed.ncbi.nlm.nih.gov/3068119/
https://pubmed.ncbi.nlm.nih.gov/26687632/
https://pubmed.ncbi.nlm.nih.gov/33775077/
https://pubmed.ncbi.nlm.nih.gov/26365362/
https://pubmed.ncbi.nlm.nih.gov/4170972/
https://pubmed.ncbi.nlm.nih.gov/4912633/
https://pubmed.ncbi.nlm.nih.gov/22242848/
https://pubmed.ncbi.nlm.nih.gov/31701183/
> e:\projects\sciencecrawler\sciencecrawler\pubmed.py(186)to_json()
    185         import ipdb; ipdb.set_trace()
--> 186         response = dict()
    187         response['id'] = self.doi.replace("doi: ","").replace("/","").replace(".","").lower().strip()

[Chloroquine]
https://pubmed.ncbi.nlm.nih.gov/8112211/
False
> e:\projects\sciencecrawler\sciencecrawler\pubmed.py(186)to_json()
    185         import ipdb; ipdb.set_trace()
--> 186         response = dict()
    187         response['id'] = self.doi.replace("doi: ","").replace("/","").replace(".","").lower().strip()

[Chlor

BdbQuit: 

In [ ]:
neo4jC = Neo4jScienceCrawler("bolt://localhost:7687",'neo4j','admin')

def save_in_neo4j(article):
    try:
        article_json = article.to_json()
        references = article.get_references()
        print("criando")
        neo4jC.create_article(article_json)
        neo4jC.change_metadado(article_json['id'],"result_pubmed",True)
        print("criando ref")
        for ref in references:
            ref_json = ref.to_json()
            neo4jC.create_connection_ref(article_json['id'],ref_json)
    except Exception as ex:
        print(ex)
    

In [ ]:
for article in articles_current_page:
    print(article.url)
    save_in_neo4j(article)

In [ ]:
article.to_json()

In [ ]:
while articles_read < total:
    try:
        time.sleep(5)
        print(articles_read)
        searchengine.get_next_page()
        articles_current_page = searchengine.get_list_articles()
        articles_read = articles_read + len(articles_current_page)
        for article in articles_current_page:
            try:
                print(article.url)
                save_in_neo4j(article)
            except Exception as ex:
                print(ex)
    except Exception as ex:
        print(ex)
      